In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm 
import pingouin as pg

# Plotting the results from 10 fold CV to use in thesis

### Data frames of all of the scalar values

In [ ]:
dice_scores = pd.DataFrame({
    "UNet" : [0.8944242596626282, 0.9031502604484558, 0.9309178590774536, 0.8758745193481445, 0.9174270629882812, 0.9058488607406616, 0.9013234972953796, 0.932015061378479, 0.9182965159416199, 0.9369605779647827],
    "UNet++" : [0.9094436168670654, 0.9018551707267761, 0.9206019639968872, 0.8962262272834778, 0.9221664667129517, 0.9176077842712402, 0.9169801473617554, 0.9323840141296387, 0.9114693403244019, 0.9373729825019836],
    "ResUNet" : [0.9031871557235718, 0.8906315565109253, 0.9184414148330688, 0.8907487988471985, 0.9177789688110352, 0.9127094149589539, 0.8939642906188965, 0.9352273941040039, 0.9235419034957886, 0.9373534917831421],
    "AttentionUNet" : [0.8931841254234314, 0.8866070508956909, 0.9292612671852112, 0.8998936414718628, 0.9133971333503723, 0.9112181663513184, 0.8928281664848328, 0.9250450134277344, 0.9134928584098816, 0.9383994340896606],
    "UNETR" : [0.8667255640029907, 0.8210844993591309, 0.914404571056366, 0.8871626853942871, 0.9002105593681335, 0.8910728693008423, 0.8798710107803345, 0.9206932783126831, 0.8904483914375305, 0.9274247884750366],
    "Swin-UNETR-v2" : [0.9063911437988281, 0.901349663734436, 0.9203064441680908, 0.9008113145828247, 0.9211304783821106, 0.9041532278060913, 0.9069981575012207, 0.9373006820678711, 0.9243558645248413, 0.9392415285110474],

})

iou = pd.DataFrame({
    "UNet" : [ 0.8180352449417114, 0.8314013481140137, 0.8720933198928833, 0.803466796875, 0.850450873374939, 0.8372943997383118, 0.8354585766792297, 0.873731255531311, 0.8540703058242798, 0.8821827173233032],
    "UNet++" : [0.8419002294540405, 0.8278276324272156, 0.8543233275413513, 0.8243061304092407, 0.8576380014419556, 0.8518285751342773, 0.8514896631240845, 0.874247670173645, 0.8426523208618164, 0.8837270736694336],
    "ResUNet" : [0.8344510197639465, 0.8133836388587952, 0.8511929512023926, 0.8165572881698608, 0.8504695892333984, 0.8460937738418579, 0.8217117190361023, 0.8790239095687866, 0.8627504110336304, 0.8828365206718445],
    "AttentionUNet" : [0.8179000616073608, 0.8065405488014221, 0.8689942359924316, 0.8288774490356445, 0.8431106805801392, 0.8432937860488892, 0.8243752717971802, 0.8624675869941711, 0.8459666967391968, 0.884682834148407],
    "UNETR" : [0.7826031446456909, 0.7325807213783264, 0.8450106382369995, 0.808093249797821, 0.8221603631973267, 0.8148366212844849, 0.7966705560684204, 0.8546293377876282, 0.8127796053886414, 0.8667184710502625],
    "Swin-UNETR-v2" : [0.8378657102584839, 0.8288360834121704, 0.8546990156173706, 0.8293713331222534, 0.8551739454269409, 0.835055947303772, 0.8376396298408508, 0.8828166723251343, 0.8640449643135071, 0.8861913681030273],
})

precision = pd.DataFrame({
    "UNet" : [0.8628487586975098, 0.9026145935058594, 0.9091062545776367, 0.9129126071929932, 0.9020407795906067, 0.9466826915740967, 0.9269645810127258, 0.9463621973991394, 0.9102045893669128, 0.9457316398620605],
    "UNet++" : [0.8846170902252197, 0.9216268062591553, 0.8914363980293274, 0.9117817878723145, 0.9025845527648926, 0.9585728049278259, 0.9572808146476746, 0.9377081990242004, 0.9008737802505493, 0.9369779825210571],
    "ResUNet" : [0.8819580078125, 0.8952786922454834, 0.900081217288971, 0.9007363319396973, 0.9086596369743347, 0.9489853978157043, 0.9292594194412231, 0.9258235096931458, 0.919293224811554, 0.948336660861969],
    "AttentionUnet" : [0.8725359439849854, 0.895419180393219, 0.9140255451202393, 0.9289263486862183, 0.9030061960220337, 0.9350361227989197, 0.9239802956581116, 0.9156268835067749, 0.9073142409324646, 0.9365703463554382],
    "UNETR" : [0.8323692083358765, 0.8433160781860352, 0.9042622447013855, 0.8766385912895203, 0.9293981790542603, 0.9442101716995239, 0.9336853623390198, 0.9251590967178345, 0.8991209864616394, 0.9106211066246033],
    "Swin-UNETR-v2" : [0.8735557198524475, 0.8985112905502319, 0.8910920023918152, 0.9065099954605103, 0.932819664478302, 0.9371129274368286, 0.9596273303031921, 0.9471668601036072, 0.917614758014679, 0.937044620513916],
})

recall = pd.DataFrame({
    "UNet" : [0.9542471170425415, 0.8953632712364197, 0.9646466970443726, 0.8739374876022339, 0.9390316009521484, 0.8674449324607849, 0.879918098449707, 0.9166195392608643, 0.9458817839622498, 0.9315346479415894],
    "UNet++" : [0.9612417817115784, 0.8776593208312988, 0.9623557329177856, 0.9038562774658203, 0.9489624500274658, 0.8817180395126343, 0.8794792890548706, 0.9268556237220764, 0.9409537315368652, 0.9387361407279968],
    "ResUNet" : [0.9503664970397949, 0.8754773139953613, 0.9464983344078064, 0.9046727418899536, 0.9368082880973816, 0.8817620873451233, 0.86075758934021, 0.9469677805900574, 0.9449235796928406, 0.9281367659568787],
    "AttentionUNet": [0.9415513277053833, 0.8733944892883301, 0.9561116099357605, 0.8887767791748047, 0.9305335879325867, 0.8902202844619751, 0.8592462539672852, 0.9361144304275513, 0.9384896159172058, 0.9420326352119446],
    "UNETR" : [0.9375269412994385, 0.8071410655975342, 0.9368401765823364, 0.9152859449386597, 0.8767292499542236, 0.8484581708908081, 0.8360879421234131, 0.9124531149864197, 0.910198450088501, 0.9465969800949097],
    "Swin-UNETR-v2" : [0.9634935259819031, 0.9009174108505249, 0.962984561920166, 0.9154780507087708, 0.9151679873466492, 0.8730837106704712, 0.8587099313735962, 0.9286044836044312, 0.9458361864089966, 0.9458362460136414]
})

comp_time_iteration = pd.DataFrame({
    "unet" : []
})

In [ ]:
#Add the folds
dice_scores['Fold'] = [f'Fold_{i+1}' for i in range(10)]
iou['Fold'] = [f'Fold_{i+1}' for i in range(10)]
precision['Fold'] = [f'Fold_{i+1}' for i in range(10)]
recall['Fold'] = [f'Fold_{i+1}' for i in range(10)]

In [ ]:
#model to calculate stats from:
model = "Swin-UNETR-v2"

print("Precision")
print(f"{precision[model].mean():.3f} +- {precision[model].std():.3f}")
print("\n")
print("Recall")
print(f"{recall[model].mean():.3f} +- {recall[model].std():.3f}")
print("\n")
print("IoU")
print(f"{iou[model].mean():.3f} +- {iou[model].std():.3f}")
print("\n")
print("Dice Scores")
print(f"{dice_scores[model].mean():.3f} +- {dice_scores[model].std():.3f}")
print("\n")



## Violine plot

In [ ]:
fig = plt.figure(figsize=(8, 5))
sns.violinplot(data=dice_scores)
plt.title("10-Fold CV Dice Scores")
plt.ylabel("Validation Dice Score")
plt.xlabel("Model")
plt.grid()
plt.show()

## Checking for normally distributed data
Using Shapiro wilk test. p>0.05 indicate normal distribution



In [ ]:
for model in dice_scores.loc[:, dice_scores.columns != "Fold"]:
    result = stats.shapiro(dice_scores[model])
    print(model)
    print(result)


Using QQ plots for visual representation

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i in range(6):
        model = dice_scores.columns[i]
        stats.probplot(dice_scores[model], dist="norm", plot=axes[i])
        axes[i].set_title(f"Q-Q Plot: {model}")
plt.tight_layout()

## RM ANOVA test

In [ ]:
dice_scores_long = dice_scores.melt(id_vars=['Fold'], var_name='Model', value_name='Dice Score')
rm_anova = pg.rm_anova(dice_scores_long, dv='Dice Score', subject='Fold', within=['Model'], correction=True)
print(rm_anova)